# 📌 Retour d'expérience installation

## Installation sur Ubuntu

*S'appliquer à chaque étape pour ne pas devoir y revenir et y perdre un temps fou.*

### Point sur la configuration système

Version d'Ubuntu (22.04.2 LTS (jammy)) :

```sh
(base)$ lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 22.04.2 LTS
Release:        22.04
Codename:       jammy
```

Version du JRE si installé :

```sh
(base)$ java -version
openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.18+10-Ubuntu-0ubuntu122.04, mixed mode, sharing)
```

Version de Python si installé :

```sh
(base)$ python --version
Python 3.9.12
```

```sh
(base)$ pyenv versions
* system (set by /home/pepper/.pyenv/version)
  3.11.2
```

Activation globale ou locale (ce dossier et ses sous-dossiers) de la version 3.11.2 :

```sh
$ pyenv global 3.11.2
```

```sh
$ pyenv local 3.11.2
```

Version de ipython installée, si installée :

```sh
(base)$ ipython --version
8.2.0
```

### Gérer les conflits d'environnement : PyEnv vs. Conda


Activation de la version 3.11.2 (`system`) suivi d'un test de version :
```sh
(base)$ pyenv global system
(base)$ python --version
Python 3.9.12
```

Ce n'est pas le résultat attendu (on s'attendait à `Python 3.11.2`).

```sh
(base)$ /usr/bin/python3 --version
Python 3.10.6
```

```sh
(base)$ which python3
/home/pepper/anaconda3/bin/python3
```

```sh
(base)$ /home/pepper/anaconda3/bin/python3 --version
Python 3.9.12
```

Manifestement il y a des conflits de versions et de gestion d'environnement entre `pyenv` et `conda`, et c'est en l'état assez confus.

```sh
(base)$ echo $PATH
~/anaconda3/bin:~/anaconda3/condabin:~/.pyenv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:usr/bin:/sbin:/bin:/usr/games:usr/local/games:/snap/bin:snap/bin
```

Interprétation du problème : une version 3.10.6 de Python est installée sur le système par défaut, mais Conda a installé une version 3.9.12 dans son propre environnement et l'a ensuite ajouté à la variable `PATH`, avec priorité sur `pyenv`.

On peut commencer par désactiver l'environnement Conda pour la session en cours. Mais il faut se poser sérieusement la question de sa désinstallation, vu que nous ne l'utilisons pas.

```sh
(base)$ conda deactivate
$ python3 --version
$ which python3
Python 3.10.6
/usr/bin/python3
```

Autre observation, le `PATH` ne contient pas le dossier `$HOME/.pyenv/shims`, donc `pyenv` n'a pas été correctement initialisé après son installation.

Commençons par désintaller Conda, pour n'avoir pas à devoir faire de la gymnastique administrative coûteuse en temps pour un non administrateur système.

```sh
$ conda env list
base * /home/pepper/anaconda3
```

Tips :
* https://stackoverflow.com/questions/22585235/python-anaconda-how-to-safely-uninstall
* https://docs.anaconda.com/free/anaconda/install/uninstall/


```sh
$ conda activate base
(base)$ conda install anaconda-clean
(base)$ anaconda-clean --yes
(base)$ rm -rf ~/anaconda3
```

Et supprimer la section Anaconda du `~/.bashrc`.

Décision finalement de ne pas le faire, `conda` pourrait être exigé par certains clients, donc il est utile de pouvoir le faire coexister avec pyenv sur mon système.

Première modification : éditer le fichier `~/.condarc` avec nano et ajouter la ligne `auto_activate_base: false
`. Cette option, qui est à `true` par défaut, active par défaut l'environnement `base` de Conda, ce qui interfère avec `pyenv init -` qui intervient ensuite dans le `.bashrc`

Réexécution du `~/.bashrc` après cette modification :
```sh
$ source ~/.bashrc
```

D'une part, l'environnement `(base)` n'est plus activé (il apparaissait depuis le début en préfixe du prompt), d'autre part :
```sh
$ echo $PATH
~/anaconda3/condabin:~/.pyenv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:usr/bin:/sbin:/bin:/usr/games:usr/local/games:/snap/bin:snap/bin
```

Le fait que `~/anaconda3/condabin` apparaisse toujours en tête de `PATH` montre (ce que l'on peut également comprendre en déchiffrant la section `conda_setup` du `~/.bashrc`) que le `~/anaconda3/etc/profile.d/conda.sh` est toujours exécuté, et qu'il produit cette modification du `PATH`. Effectivement, il le fait dans sa dernière section `PATH="../condabin${PATH:+"${PATH}"}"`

Après pas mal d'investigation (et une demi-journée crâmée) pour tenter de faire coexister proprement conda et pyenv, voici où j'en suis :

* Conda eexcute un hook de shell, un script qu'il génère puis exécute, donc sur lequel on n'a ni la main, ni de visibilité sur le contenu :
* génération du script : `__conda_setup="$('/home/pepper/anaconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"`
* exécution du script (s'il a été produit sans erreur : `if [ $? -eq 0 ]`) : `eval "$__conda_setup"`

Je jette l'éponge, faute de temps. Je passe la section conda de `.bashrc` en commentaire.

Au moins, cette demi-journée m'aura-t-elle un peu réchauffé sur le `shell` et sur l'utilisation de `nano`.

On redémarre de la vérification des versions :

```sh
$ python3 --version
$ which python3
Python 3.10.6
/usr/bin/python3
```

Activation de la version 3.11.2 (`system`) suivi d'un test de version :
```sh
(base)$ pyenv global system
(base)$ python --version
Python 3.9.12
```

Je viens de comprendre, pour un coût de 5h, que j'ai juste mal installé pyenv :

Dans le `bashrc`, j'ai un `eval "(pyenv init -)"` qui se contente de générer le script mais ne l'excute pas.

Fix : `eval "$(pyenv init -)"`

### Mise à jour et redémarrage du système

```sh
$ sudo apt update && sudo apt -y full-upgrade
```
* `sudo apt update` : met à jour la liste des paquets disponibles à partir des sources de logiciels configurées sur le système. Il ne met pas à jour les paquets eux-mêmes.
* `sudo apt -y full-upgrade` : met à jour les paquets existants sur le système en installant les dernières versions disponibles. L'option -y répond automatiquement « oui » à toutes les questions de confirmation qui peuvent apparaître pendant le processus.
* Le `&&` est un opérateur logique qui permet d'exécuter plusieurs commandes en même temps, l'une après l'autre, à condition que la commande précédente s'exécute avec succès. Dans cet exemple, la seconde commande `sudo apt -y full-upgrade` ne sera exécutée que si la commande `sudo apt update` s'exécute avec succès.


```sh
$ [ -f /var/run/reboot-required ] && sudo reboot -f
```

* `[ -f /var/run/reboot-required ]` : vérifie si le fichier /var/run/reboot-required existe. Ce fichier est créé lorsqu'un paquet est installé qui nécessite un redémarrage pour prendre effet.
* `&&` : l'opérateur logique « et ». Si la commande précédente ([ -f /var/run/reboot-required ]) est vraie (c'est-à-dire que le fichier /var/run/reboot-required existe), alors la commande suivante (sudo reboot -f) est exécutée.
* `sudo reboot -f` : redémarre le système en forçant le redémarrage sans demander de confirmation.

### Installation des prérequis

La doc de PySpark indique d'installer ces prérequs :

```sh
sudo apt-get install build-essential checkinstall
sudo apt-get install libreadline-gplv2-dev libncursesw5-dev libssl-dev libsqlite3-dev tk-dev libgdbm-dev libc6-dev libbz2-dev
sudo apt-get install python
sudo easy_install pip
sudo pip install ipython
```

On peut commencer par faire un point sur ce qui est déjà installé ou non :

```sh
dpkg-query -l | grep checkinstall           # Suivi d'installations
dpkg-query -l | grep build-essential        # Compilation de programmes
dpkg-query -l | grep libreadline-gplv2-dev  # édition interactive d'entrées en ligne de commande
dpkg-query -l | grep libncursesw5-dev       # interfaces utilisateurs textuelles pour terminaux
dpkg-query -l | grep libssl-dev             # Open SSL
...
ii build-essentiel 12.9ubuntu3 amd64 Informational list of builde-essentiel packages
ii libssl-dev 3.0.2-0ubuntu1.9 amd64 Secure Sockets Layer toolkit - development files
```

Point rapide sur ces paquets :
* **`build-essential`** est un ensemble de paquets contenant des outils nécessaires à la compilation des programmes sur Ubuntu.
* **`libreadline-gplv2-dev`** contient les fichiers de développement nécessaires pour utiliser la bibliothèque `readline` dans les programmes.
* **`libncursesw5-dev`** contient les fichiers de développement pour la bibliothèque `ncurses`, qui est utilisée pour créer des interfaces utilisateurs textuelles pour les terminaux.
* **`libssl-dev`** contient les fichiers de développement pour la bibliothèque `OpenSSL`, qui est utilisée pour la cryptographie dans de nombreux programmes.
* **`libsqlite3-dev`** contient les fichiers de développement pour la bibliothèque `SQLite3`, qui est utilisée pour la gestion de base de données.
* **`tk-dev`** contient les fichiers de développement pour le toolkit `Tk`, qui est utilisé pour créer des interfaces graphiques pour les applications.
* **`libgdbm-dev`** contient les fichiers de développement pour la bibliothèque `GDBM`, qui est utilisée pour stocker des paires clé/valeur dans des fichiers.
* **`libc6-dev`** contient les fichiers de développement pour la bibliothèque `C`, qui est la bibliothèque standard de manipulation de chaînes, de fichiers et d'entrées/sorties pour les programmes en `C`.
* **`libbz2-dev`** contient les fichiers de développement pour la bibliothèque `Bzip2`, qui est utilisée pour la compression de fichiers.
* **`checkinstall`** est un outil qui permet de créer des paquets Debian à partir des programmes installés depuis les sources. Cela facilite la gestion des programmes installés sur le système, permettant de les installer, les désinstaller et les mettre à jour plus facilement.

`PySpark` utilise la bibliothèque `Py4J` pour établir une communication entre Python et le processus JVM de Spark. `Py4J` utilise la bibliothèque `Readline` pour l'interpréteur Python interactif, et la bibliothèque `Ncurses` pour l'interface utilisateur textuelle de son outil `py4j-asyncio`. Ces deux librairies sont également nécessaires pour améliorer l'expérience utilisateur avec le terminal interactif `iPython`.

La bibliothèque `SQLite3` est utilisée par `PySpark` pour stocker des informations sur les tâches en cours d'exécution dans la base de données intégrée à `Spark`. En outre, `PySpark` peut utiliser la bibliothèque `Bzip2` pour la compression des données.

Dans le contexte de l'installation de `PySpark`, `checkinstall` n'est pas strictement nécessaire, car l'installation de `PySpark` se fait généralement à partir de fichiers binaires pré-compilés, plutôt que depuis les sources. Cependant, l'utilisation de `checkinstall` peut être utile si vous souhaitez installer d'autres programmes à partir des sources à l'avenir.

```sh
...
le paquet "libreadline-gplv2-dev" n'a pas de version susceptible d'être installée.
```

On fait quoi pour ne pas avoir des problèmes plus tard ?

Si le paquet `libreadline-gplv2-dev` n'a pas de version disponible, il est possible que le nom du paquet ait changé ou qu'il ne soit plus disponible dans les sources de la distribution Ubuntu.

Prenons une alternative à la bibliothèque `readline`, comme la bibliothèque GNU readline.

```sh
sudo apt-get install libreadline-dev
```

Il sera peut-être nécessaire, plus tard dans l'installation, de modifier les configurations de `Py4J` et de `PySpark` pour qu'ils utilisent la nouvelle bibliothèque.


On nous demande d'installer un `pip` pour le système entier (hors environnement virtuel)

```sh
sudo easy_install pip
```

Restons sur la version liée à un environnement :
```sh
sudo apt-get install python3-pip 
```

Nous verrons plus tard si cela pose un problème.


Installation de `iPython`

Dans le contexte `PySpark`, `iPython` qui joue habituellement le rôle de back-office nécessaire pour l'utilisateur de notebooks va jouer son rôle premier qui est de fournir un terminal de substitution au terminal par défaut. Ce terminal plus évolué permet de bénéficier de fonctionnalités supplémentaires telles que la coloration syntaxique, l'auto-complétion et l'historique des commandes.

```sh
$ sudo pip install ipython 
```

### Installation de Java

Version du JRE si installé :

```sh
$ java -version
openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.18+10-Ubuntu-0ubuntu122.04, mixed mode, sharing)
```

Sinon, cf. doc préconisée :
```sh
sudo apt install curl mlocate default-jdk -y
```

et cf. manuel `PySpark` :
```sh
sudo apt-add-repository ppa:webupd8team/java
sudo apt-get update
sudo apt-get install oracle-java8-installer
```

### Installation d'`Apache Spark`

A ne pas confondre avec `PySpark` qui est l'API pour interagir avec `Apache Spark` depuis un environnement de développement Python (comme alternative à `Java` ou `Scala`).

Deux étapes :
* télécharger et installer
* configurer

Cf. doc préconisée :
```sh
wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
tar xvf spark-3.2.1-bin-hadoop3.2.tgz
sudo mv spark-3.2.1-bin-hadoop3.2/ /opt/spark 
```

et cf. manuel `PySpark` :
* Download: You can get the Pre-built Apache Spark™ from [**Download Apache Spark™**](https://spark.apache.org/downloads.html).
* Unpack: Unpack the Apache Spark™ to the path where you want to install the Spark.

On va rester dans une approche wget etc, mais en prenant la dernière version, la 3.4.0 (du 13 avril 2023).

https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz

```sh
wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
tar xvf spark-3.4.0-bin-hadoop3.tgz
sudo mv spark-3.4.0-bin-hadoop3/ /opt/spark 
```

Attention, si on visite le dossier de téléchargement `https://dlcdn.apache.org/spark/spark-3.4.0/`, on constate qu'il y a plusieurs archives qui correspondent à des contenus de différentes natures :
* le noyau d'infrastructure Spark:
    * qui n'embarque pas Hadoop (qui pourrait être déjà installé sur le système)
    * qui l'embarque
        * version validée pour fonctionner avec Scala 2.13
        * version qui fonctionne avec une version de Scala antérieure
* des packages d'API pour interagir avec cette infrastructure qu'il faut donc avoir préalablement installée :
    * SparkR : programmation R
    * PySpark : programmation Python

La distinction entre les distributions spark et spark_witout Hadoop n'est pas immédiatement claire :
* `spark-3.4.0-bin-without-hadoop` ne contient pas les bibliothèques Hadoop nécessaires à l'exécution de Spark sur un cluster Hadoop existant. Il est destiné aux utilisateurs qui souhaitent exécuter Spark sur un cluster non-Hadoop ou qui souhaitent intégrer Spark avec une version Hadoop spécifique qu'ils ont déjà installée.
* `spark-3.4.0`, quant à lui, est livré avec des bibliothèques Hadoop pré-compilées pour Hadoop 3.2 et n'a donc pas besoin d'une installation Hadoop séparée pour fonctionner. Il est destiné aux utilisateurs qui veulent une distribution Spark autonome, prête à l'emploi.

Nous utiliserons la version `Spark + Hadoop` et `PySpark` pour une interaction en Python.

Le répertoire `/opt` est une convention de l'*Unix Filesystem Hierarchy Standard* qui permet de stocker des données d'applications supplémentaires qui ne font pas partie du système d'exploitation. Les applications tierces peuvent être installées dans ce dossier sans interférer avec les fichiers système existants. Dans le cas de `Spark`, il est recommandé de stocker l'installation dans `/opt` pour des raisons de sécurité et de gestion de version.

`https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz` fait 370M

C'est parti :

```sh
wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
tar xvf spark-3.4.0-bin-hadoop3.tgz
sudo mv spark-3.4.0-bin-hadoop3/ /opt/spark 
```

Ce n'est pas précisé, mais :
```sh
rm spark-3.4.0-bin-hadoop3.tgz 
```
n'est pas inutile.

Reste à configurer le `.bashrc` pour la mise à jour du `PATH`:

```sh
nano ~/.bashrc
```

Ajout des lignes
```sh
export SPARK_HOME=/opt/spark
export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
```

et mise à jour de l'environnement du terminal :
```sh
source ~/.bashrc
```

Test de bon fonctionnement, avec le lancement d'un serveur maître autonome (Standalone Master Server) :
```sh
$ start-master.sh 
starting org.apache.spark.deploy.master.Master, logging to /opt/spark/logs/spark-root-org.apache.spark.deploy.master.Master-1-<Ma machine>.out
```

Le processus écoute sur le port TCP 8080 :
```sh
$ sudo ss -tunelp | grep 8080
tcp   LISTEN  0       1                           *:8080                *:*      users:(("java",pid=15356,fd=307)) ... sk:5 v6only:0 <-> 
```

Ouverture d'un navigateur sur l'URL `http://localhost:8080` :

L'intreface Web de supervision s'affiche : `Spark Master at spark://SalviaDivinorum:7077`

Mon URL spark est `spark://SalviaDivinorum:7077`

Cette URL correspond à l'adresse du serveur Spark utilisée par le client Spark pour communiquer avec le cluster Spark. Plus précisément, cette URL spécifie le mode de connexion "standalone", utilisé pour exécuter des applications Spark sur un cluster autonome.

La partie "SalviaDivinorum" correspond au nom de la machine sur laquelle le serveur Spark est exécuté et le port 7077 est le port par défaut sur lequel le serveur Spark écoute les connexions entrantes. Si vous avez spécifié un port différent lors de la configuration du serveur Spark, alors le numéro de port dans l'URL sera différent.

**NB** les url en général ne sont pas sensibles à la casse, mais il faudra s'assurer que l'emploi de `spark://salviadivinorum:7077` ne pose pas de problème.

On va pouvoir le faire de suite avec le test de client.

Lancement un Spark Worker Process esclave :

```sh
$ start-slave.sh spark://ubuntu:7077
starting org.apache.spark.deploy.worker.Worker, logging to /opt/spark/logs/spark-root-org.apache.spark.deploy.worker.Worker-1-<Ma machine>.out
```

Pas de problème avec l'URL, le Worker client est lancé.

Dan l'UI, le worker est bien visible.

Lancement du `Spark shell`

```sh
$ spark-shell
...
scala> ... si on ne maîtrise pas Scala, on a besoin de PySpark !^^
```

Le Spark Shell pour Scala est un REPL (Read-Eval-Print Loop) qui s'exécute dans le terminal standard du système d'exploitation, et non dans une interface comme `iPython`. Pour bénéficier du terminal `iPython` avec `Spark`, il faut utiliser `PySpark`.

Deux bonnes raisons de passer à la dernière étape d'installation avec `PySpark`.

Pour fermer proprement les processus maître et esclave lancés précédemment :

Pas cela, c'est déprécié :
```sh
$ SPARK_HOME/sbin/stop-slave.sh
$ SPARK_HOME/sbin/stop-master.sh
```

Mais cela:
```sh
$ SPARK_HOME/sbin/stop-worker.sh
$ SPARK_HOME/sbin/stop-master.sh
```

### Installation de PySpark

Nous souhaitons disposer de toutes les fonctionnalités supplémentaires qui ne font pas partie de la version de base :
* **`pyspark[sql]`** : ajoute le support pour les DataFrames SQL, ce qui permet d'utiliser SQL pour interagir avec les données.
* **`pyspark[pandas_on_spark]`** : ajoute le support pour l'utilisation de Pandas avec Spark, ce qui permet de travailler avec des données Pandas sur un cluster Spark.
* **`pyspark[plotly]`** : ajoute le support pour la visualisation avec Plotly, une bibliothèque de visualisation interactive.
* **`pyspark[connect]`** : ajoute le support pour la connexion à des bases de données externes telles que Cassandra, HBase, etc.

Installation complète :

Attention, même si cela est tentant, ne pas ajouter d'espace derrière les virgules.

```sh
$ pip install pyspark
$ pip install pyspark[sql,pandas_on_spark,plotly,connect]
```

Lancement du shell PySpark :

```sh
$ pyspark
```

Lancement avec le terminal évolué `IPython` :

Cela reste à cette heure une inconnue même si j'ai quelques tips ici :

https://stackoverflow.com/questions/31862293/how-to-load-ipython-shell-with-pyspark

Ici https://github.com/apache/spark/blob/master/bin/pyspark

Il y un message plutôt explicite :

```sh
# Fail noisily if removed options are set
if [[ -n "$IPYTHON" || -n "$IPYTHON_OPTS" ]]; then
  echo "Error in pyspark startup:"
  echo "IPYTHON and IPYTHON_OPTS are removed in Spark 2.0+. Remove these from the environment and set PYSPARK_DRIVER_PYTHON and PYSPARK_DRIVER_PYTHON_OPTS instead."
  exit 1
fi
```

Comme c'est du confort, nous étudierons les deux variables `PYSPARK_DRIVER_PYTHON` et `PYSPARK_DRIVER_PYTHON_OPTS` à une prochaine itération.

Installation sur Ubuntu complétée !

## Installation sur Windows

*S'appliquer à chaque étape pour ne pas devoir y revenir et y perdre un temps fou.*

https://sparkbyexamples.com/spark/apache-spark-installation-on-windows/

La section 3.3 de [Learning Apache Spark with Python](https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf) renvoie directement sur ce tutoriel : [Getting Started with PySpark on Windows](http://deelesh.github.io/pyspark-windows.html)

Ce tutoriel comme le précédent, qui me semblent un peu datés, m'invitent à installer Winutils :

* **Spark with winutils.exe on Windows** :

*Many beginners think Apache Spark needs a Hadoop cluster installed to run but that’s not true, Spark can run on AWS by using S3, Azure by using blob storage without Hadoop and HDFS e.t.c.*

*To run Apache Spark on windows, you need winutils.exe as it uses POSIX like file access operations in windows using windows API.*

*winutils.exe enables Spark to use Windows-specific services including running shell commands on a windows environment.*

*Download winutils.exe for Hadoop 2.7 and copy it to %SPARK_HOME%\bin folder. Winutils are different for each Hadoop version hence download the right version based on your Spark vs Hadoop distribution from https://github.com/steveloughran/winutils*


Je me demande si c'est vraiment une bonne idée d'entrer dans une telle usine à gaz.

En outre, maintenant, sous Windows, on a WSL, donc la distribution Linux de son choix, pourquoi donc ne pas en tirer parti ?

L'époque du dual boot est passée, en je peux donc en pratique :
* Installer Spark dans WSL
* Partager des fichiers entre la partie Windows et la partie Linux du système.

En d'autres termes :
* Je peux continuer à développer dans mon VS Code préféré côté Windows.
* Et tester sur mon Spark local, les mêmes fichiers, juste en changeant de fenêtre.
* Rien ne m'empêche d'installer PySpark également côté Windows pour :
  * Bénéfichier du check pylance des imports et des dosctrings, même si je peux exécuter directement en local
  * Pour directement déployer dans le cloud ce que j'ai ainsi pu tester en local.


### Point sur la configuration système

Version de Windows - depuis PowerShell :

```sh
systeminfo   # | Select-String "^OS Name","^OS Version"
Nom de l’hôte:                              BELLADONNA
Nom du système d’exploitation:              Microsoft Windows 11 Famille
Version du système:                         10.0.22621 N/A build 22621
Fabricant du système d’exploitation:        Microsoft Corporation
Configuration du système d’exploitation:    Station de travail autonome
Type de build du système d’exploitation:    Multiprocessor Free
Propriétaire enregistré:                    franck.lepoivre@platypus.academy
Organisation enregistrée:                   N/A
Identificateur de produit:                  00325-81958-50222-AAOEM
Date d’installation originale:              05/02/2023, 20:34:23
Heure de démarrage du système:              06/05/2023, 16:17:25
Fabricant du système:                       Acer
Modèle du système:                          Nitro AN517-51
Type du système:                            x64-based PC
Processeur(s):                              1 processeur(s) installé(s).
                                            [01] : Intel64 Family 6 Model 158 Stepping 10 GenuineIntel ~2592 MHz
Version du BIOS:                            Insyde Corp. V1.31, 29/06/2020
Répertoire Windows:                         C:\WINDOWS
Répertoire système:                         C:\WINDOWS\system32
Périphérique d’amorçage:                    \Device\HarddiskVolume1
Option régionale du système:                fr;Français (France)
Paramètres régionaux d’entrée:              fr;Français (France)
Fuseau horaire:                             (UTC+01:00) Bruxelles, Copenhague, Madrid, Paris
Mémoire physique totale:                    16 221 Mo
Mémoire physique disponible:                3 434 Mo
Mémoire virtuelle : taille maximale:        65 373 Mo
Mémoire virtuelle : disponible:             44 513 Mo
Mémoire virtuelle : en cours d’utilisation: 20 860 Mo
Emplacements des fichiers d’échange:        C:\pagefile.sys
Domaine:                                    WORKGROUP
Serveur d’ouverture de session:             \\BELLADONNA
Correctif(s):                               4 Corrections installées.
                                            [01]: KB5022497
                                            [02]: KB5012170
                                            [03]: KB5025239
                                            [04]: KB5025749
Carte(s) réseau:                            2 carte(s) réseau installée(s).
                                            [01]: Killer E2500 Gigabit Ethernet Controller
                                                  Nom de la connexion : Ethernet
                                                  État :                Support déconnecté
                                            [02]: Intel(R) Wi-Fi 6 AX200 160MHz
                                                  Nom de la connexion : Wi-Fi
                                                  DHCP activé :         Oui
                                                  Serveur DHCP :        192.168.1.254
                                                  Adresse(s) IP
                                                  [01]: 192.168.1.168
                                                  [02]: fe80::307a:8990:2f21:b062
                                                  [03]: 2a01:e0a:272:1f00:91ba:73cb:9a8b:6729
                                                  [04]: 2a01:e0a:272:1f00:6c68:bf4:ce90:39fa
                                                  [05]: 2a01:e0a:272:1f00:607d:4e80:fbb0:a6dd
                                                  [06]: 2a01:e0a:272:1f00:fabb:fe7d:63ce:c470
Configuration requise pour Hyper-V:         Un hyperviseur a été détecté. Les fonctionnalités nécessaires à Hyper-V ne seront pas affichées.
...
```

Quelles distributions Linux installée sur WSL ? Depuis PowerShell :

```sh
wsl --list --verbose
  NAME      STATE           VERSION
* Ubuntu    Running         2
```

Depuis WSL :

```sh
cat /etc/os-release
NAME="Ubuntu"
VERSION="20.04.4 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.4 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal
```

Version du JRE si installé :

Côté Ubuntu/WSL :
```sh
$ java --version
openjdk 11.0.15 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.20.04.1, mixed mode, sharing)
```

Une mise à jour ne serait pas inutile :

```sh
sudo apt-get update && sudo apt-get dist-upgrade
```

```sh
$ java --version
openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
```

C'est mieux!

```sh
$ python3 --version
Python 3.8.10
$ which python3
/usr/bin/python3
$ pyenv versions
/mnt/c/Users/franc/.pyenv/pyenv-win/bin/pyenv: 3: cygpath: not found
/mnt/c/Users/franc/.pyenv/pyenv-win/bin/pyenv: 3: exec: cmd: not found
```

Installation `pyenv` : https://kfields.netlify.app/blog/pyenv_on_ubuntu_22

```sh
sudo apt-get update; sudo apt-get install make build-essential libssl-dev zlib1g-dev libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm libncursesw5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev liblzma-dev
curl https://pyenv.run | bash
```

Maj. `.bashrc`:

```sh
$ nano ~/.bashrc
# Ajout à la fin de :
export PATH="$HOME/.pyenv/bin:$PATH"
eval "$(pyenv init --path)"
eval "$(pyenv virtualenv-init -)"
```

Prise en compte dans la session en cours:

```sh
$ source ~/.bashrc
```

Installation de Python 3.11 dans Pyenv et activation globale.

```sh
$ pyenv versions
* system (set by ~/.pyenv/version)
$ pyenv install 3.11
$ pyenv install 3.11
Downloading Python-3.11.3.tar.xz...
-> https://www.python.org/ftp/python/3.11.3/Python-3.11.3.tar.xz
Installing Python-3.11.3...
Installed Python-3.11.3 to ~/.pyenv/versions/3.11.3
$ pyenv versions
* system (set by ~/.pyenv/version)
  3.11.3
$ python3 --version
Python 3.8.10
$ which python3
~/.pyenv/shims/python3
$ pyenv global 3.11
$ python3 --version
Python 3.11.3
```

Version de ipython installée, si installée :

```sh
(base)$ ipython --version
8.2.0
```

### Installation des packages nécessaires

```sh
$ sudo apt-get install build-essential checkinstall
$ sudo apt-get install libreadline-gplv2-dev    # Hum, ici pas de pb, et c'est libreadline-dev qui a été remplacée
$ sudo apt-get install libncursesw5-dev libssl-dev libsqlite3-dev tk-dev libgdbm-dev libc6-dev libbz2-dev
$ sudo pip install ipython
```

### Installation d'`Apache Spark`

C'est parti cf. plus haut, avec un petit déplacement préalable vers ~ (c'est plus propre).

```sh
cd ~
wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
tar xvf spark-3.4.0-bin-hadoop3.tgz
sudo mv spark-3.4.0-bin-hadoop3/ /opt/spark
rm spark-3.4.0-bin-hadoop3.tgz  
```

Reste à configurer le `.bashrc` pour la mise à jour du `PATH`:

```sh
nano ~/.bashrc
# Ajout des lignes
export SPARK_HOME=/opt/spark
export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
# mise à jour de l'environnement du terminal :
source ~/.bashrc
```

Test de bon fonctionnement, avec le lancement d'un serveur maître autonome (Standalone Master Server) :
```sh
$ start-master.sh 
starting org.apache.spark.deploy.master.Master, logging to /opt/spark/logs/spark-root-org.apache.spark.deploy.master.Master-1-<Ma machine>.out
```

Le processus écoute sur le port TCP 8080 :
```sh
$ sudo ss -tunelp | grep 8080
tcp   LISTEN  0       1                           *:8080                *:*      users:(("java",pid=15356,fd=307)) ... sk:5 v6only:0 <-> 
```

Ouverture d'un navigateur sur l'URL `http://localhost:8080` :

Reste à installer un navigateur (merci à wslg!):

```sh
$ sudo apt install firefox
$ firefox   # --no-remote (pour éviter err : [ERROR glean_core] Error setting metrics feature config: Json(Error("EOF while parsing a value", line: 1, column: 0)))
```

Ok : `spark://Belladonna.localdomain:7077`

Peut-on former un réseau local pour accéder depuis Windows au port HTTP d'Ubuntu/WSL ?

Côté Ubuntu/WSL : IP **172.28.176.216**
```sh
$ sudo apt install net-tools
$ ifconfig
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.28.176.216  netmask 255.255.240.0  broadcast 172.28.191.255
        inet6 fe80::215:5dff:fefc:75ad  prefixlen 64  scopeid 0x20<link>
        ether 00:15:5d:fc:75:ad  txqueuelen 1000  (Ethernet)
        RX packets 559833  bytes 825660163 (825.6 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 58719  bytes 4282355 (4.2 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 290  bytes 438843 (438.8 KB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 290  bytes 438843 (438.8 KB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0
```

Côté PowerShell/Windows : IP **172.28.176.1**
```sh
$ ipconfig

Configuration IP de Windows


Carte Ethernet Ethernet :

   Statut du média. . . . . . . . . . . . : Média déconnecté
   Suffixe DNS propre à la connexion. . . :

Carte réseau sans fil Connexion au réseau local* 1 :

   Statut du média. . . . . . . . . . . . : Média déconnecté
   Suffixe DNS propre à la connexion. . . :

Carte réseau sans fil Connexion au réseau local* 3 :

   Statut du média. . . . . . . . . . . . : Média déconnecté
   Suffixe DNS propre à la connexion. . . :

Carte réseau sans fil Wi-Fi :

   Suffixe DNS propre à la connexion. . . :
   Adresse IPv6. . . . . . . . . . . . . .: 2a01:e0a:272:1f00:fabb:fe7d:63ce:c470
   Adresse IPv6 temporaire . . . . . . . .: 2a01:e0a:272:1f00:607d:4e80:fbb0:a6dd
   Adresse IPv6 temporaire . . . . . . . .: 2a01:e0a:272:1f00:6c68:bf4:ce90:39fa
   Adresse IPv6 temporaire . . . . . . . .: 2a01:e0a:272:1f00:91ba:73cb:9a8b:6729
   Adresse IPv6 de liaison locale. . . . .: fe80::307a:8990:2f21:b062%15
   Adresse IPv4. . . . . . . . . . . . . .: 192.168.1.168
   Masque de sous-réseau. . . . . . . . . : 255.255.255.0
   Passerelle par défaut. . . . . . . . . : fe80::72fc:8fff:fe50:f53c%15
                                       192.168.1.254

Carte Ethernet vEthernet (WSL) :

   Suffixe DNS propre à la connexion. . . :
   Adresse IPv6 de liaison locale. . . . .: fe80::c684:16ec:dac1:9bf2%44
   Adresse IPv4. . . . . . . . . . . . . .: 172.28.176.1
   Masque de sous-réseau. . . . . . . . . : 255.255.240.0
   Passerelle par défaut. . . . . . . . . :
```


Windows Defender :


Ouverture des ports Spark/Ubuntu/WSL pour permettre un accès depuis un navigateur depuis la même machine (ou une autre du réseau privé) à l'UI de Spark.
1. Ouverture de Windows Defender.
2. Accès à la section 'Paramètres avancés' dans le volet de gauche
3. Cliquer sur 'Règles de trafic entrant' dans le volet de gauche, puis sur 'Nouvelle règle' dans le volet de droite.
4. Sélectionner 'Port' et cliquer sur 'Suivant'.
5. Sélectionner 'TCP' et entrer "8080, 7077" dans le champ 'Ports locaux'. Cliquer sur 'Suivant'.
6. Sélectionner 'Autoriser la connexion' et cliquer sur 'Suivant'.
7. Sélectionnez les profils pour lesquels autoriser la connexion ('Privé') et cliquez sur 'Suivant'.
8. Nommer la règle (par exemple, 'Spark Ports'), ajouter un descriptif et cliquer sur 'Terminer'.

*Ouverture des ports Spark/Ubuntu/WSL pour permettre un accès depuis un navigateur depuis la même machine (ou une autre du réseau privé) à l'UI de Spark.*

http://172.28.176.216:8080/ depuis un navigateur Windows. Great job, ça fonctionne!


Les adresse de mon réseau privé attribuées par mon routeur sont en 192.168.1.*.
Les connexions passent principalement par les cartes Wifi.
Mais les machines sont évidemment équipées de cartes ethernet filaires, même si elles ne sont pas utilisées.
Les deux adresses associées par WSL le sont à la même carte ethernet, et sont respectivement 172.28.176.1 et 172.28.176.216 : cela signifie que WSL forme son propre réseau local entre les différentes instances de machines (de systèmes) qui coexistent sur la même machine, et que ce réseau privé est donc distinct de mon réseau domestique.


How do I allow Windows Defender to allow access via HTTP to machines in my home network?
https://social.technet.microsoft.com/Forums/en-US/7076fce9-3a42-48c3-b2aa-2c09fc3a064a/how-do-i-allow-windows-defender-to-allow-access-via-http-to-machines-in-my-home-network?forum=win10itpronetworking


Lancement un Spark Worker Process esclave :

```sh
$ start-slave.sh spark://ubuntu:7077
starting org.apache.spark.deploy.worker.Worker, logging to /opt/spark/logs/spark-root-org.apache.spark.deploy.worker.Worker-1-<Ma machine>.out
```

Pas de problème avec l'URL, le Worker client est lancé.

Dans l'UI, le worker est bien visible.

Lancement du `Spark shell`

```sh
$ spark-shell
...
scala> ... si on ne maîtrise pas Scala, on a besoin de PySpark !^^
```

Le Spark Shell pour Scala est un REPL (Read-Eval-Print Loop) qui s'exécute dans le terminal standard du système d'exploitation, et non dans une interface comme `iPython`. Pour bénéficier du terminal `iPython` avec `Spark`, il faut utiliser `PySpark`.

Deux bonnes raisons de passer à la dernière étape d'installation avec `PySpark`.

Pour fermer proprement les processus maître et esclave lancés précédemment :

Pas cela, c'est déprécié :
```sh
$ SPARK_HOME/sbin/stop-slave.sh
$ SPARK_HOME/sbin/stop-master.sh
```

Mais cela:
```sh
$ SPARK_HOME/sbin/stop-worker.sh
$ SPARK_HOME/sbin/stop-master.sh
```

# Apache Arrow

https://arrow.apache.org/docs/index.html

https://pypi.org/project/pyarrow/


Apache Arrow est une plate-forme de développement pour l'analyse en mémoire vive. Elle contient un ensemble de technologies qui permettent aux systèmes de traitement de données volumineuses de traiter et de déplacer rapidement les données. Elle spécifie un format de mémoire colonnaire normalisé indépendant du langage pour les données plates et hiérarchiques, organisé pour des opérations d'analyse efficaces sur le matériel moderne.

Le projet développe une collection multilingue de bibliothèques pour résoudre des problèmes de systèmes liés au traitement de données analytiques en mémoire vive. Cela inclut des sujets tels que :
* Partage de mémoire et déplacement de données basés sur RPC sans copie
* Lecture et écriture de formats de fichiers (comme CSV, Apache ORC et Apache Parquet)
* Analyse en mémoire vive et traitement de requêtes

On pourrait penser à tort qu'Apache Arrow est la brique technologique qui permet à Hadoop de passer en mémoire vive les concepts de calcul d'Hadoop (paradigme du diviser pour régner implémenté en le framwork Map/Reduce). Il n'en est rien.

Apache Arrow est une technologie distincte de Spark, mais elle peut être utilisée en conjonction avec Spark pour améliorer les performances de traitement de données in-memory. Apache Arrow fournit une représentation efficace et standardisée des données en mémoire, indépendante de tout langage de programmation, qui peut être utilisée pour faciliter le partage de données entre différents systèmes de traitement de données. Cela permet de réduire les coûts de traitement et de stockage des données en évitant les conversions de formats de données coûteuses. Spark utilise la technologie de l'in-memory computing pour exécuter des tâches de traitement de données plus rapidement que Hadoop MapReduce, mais Apache Arrow peut être utilisé pour améliorer encore ces performances.

Apache Arrow et Spark sont tous deux basés sur des concepts similaires de traitement de données, mais ils sont des technologies complémentaires plutôt que concurrentes. Apache Arrow fournit une plate-forme pour le traitement de données en mémoire et permet le partage de données entre différents systèmes, tandis que Spark fournit un framework pour le traitement distribué de données. Ils peuvent être utilisés ensemble pour obtenir des performances améliorées et une intégration plus transparente des données entre différents systèmes.

# Point sur les ressources matérielles disponibles

Il s'agit de dimensionner et de bien partager entre le noeud maître et l'executor.

## Côté Windows

### Processeurs (CPU et GPUs)

Pour connaître le nombre de coeurs et leur capacité à l'hyperthreading sur Windows, il faut ouvrir le gestionnaire des tâches (`Ctrl+Shift+Esc`), puis aller dans l'onglet "Performances". On peut y voir le nombre de coeurs physiques et logiques du processeur.

**Belladonna** :
* CPU - Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
    * Vitesse de base : 2,59 GHz
    * Sockets : 1
    * Coeurs : 6
    * Processeurs logiques : 12
    * Virtualisation : Activée
    * Cache de niveau 1 : 384 Ko
    * Cache de niveau 2 : 1,5 Mo
    * Cache de niveau 3 : 12,0 Mo
* GPU 0 - Intel(R) UHD Graphics 630
    * Mémoire partagée : 7,9 Go
* GPU 1 - NVIDIA GeForce RTX 2060
    * Mémoire dédiée : 6 Go
    * Mémoire partagée : 7,9 Go


On peut aussi utiliser la commande PowerShell suivante :

```sh
Get-WmiObject -Class Win32_Processor | Select-Object -Property Name, NumberOfCores, NumberOfLogicalProcessors
Name                                     NumberOfCores NumberOfLogicalProcessors
----                                     ------------- -------------------------
Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz             6                        12
```

Depuis Ubuntu :

```sh
lscpu
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          12
On-line CPU(s) list:             0-11
Thread(s) per core:              2
Core(s) per socket:              6
Socket(s):                       1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           158
Model name:                      Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Stepping:                        10
CPU MHz:                         2592.007
BogoMIPS:                        5184.01
Virtualization:                  VT-x
Hypervisor vendor:               Microsoft
Virtualization type:             full
L1d cache:                       192 KiB
L1i cache:                       192 KiB
L2 cache:                        1.5 MiB
L3 cache:                        12 MiB
Vulnerability Itlb multihit:     KVM: Mitigation: VMX disabled
Vulnerability L1tf:              Mitigation; PTE Inversion; VMX conditional cache flushes, SMT vulnerable
Vulnerability Mds:               Vulnerable: Clear CPU buffers attempted, no microcode; SMT Host state unknown
Vulnerability Meltdown:          Mitigation; PTI
Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl and seccomp
Vulnerability Spectre v1:        Mitigation; usercopy/swapgs barriers and __user pointer sanitization
Vulnerability Spectre v2:        Mitigation; Full generic retpoline, IBPB conditional, IBRS_FW, STIBP conditional, RSB f
                                 illing
Vulnerability Srbds:             Unknown: Dependent on hypervisor status
Vulnerability Tsx async abort:   Not affected
Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxs
                                 r sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology cpui
                                 d pni pclmulqdq vmx ssse3 fma cx16 pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c r
                                 drand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp tpr_
                                 shadow vnmi ept vpid ept_ad fsgsbase bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap c
                                 lflushopt xsaveopt xsavec xgetbv1 xsaves flush_l1d arch_capabilities
```


### Mémoire

Pour connaître la capacité mémoire réelle disponible sur Windows, c'est également par le gestionnaire des tâches (`Ctrl+Shift+Esc`), onglet "Performances". On peut y voir la quantité de mémoire physique installée sur le système.

**Belladonna** :
    * 16 Go
    * Facteur de forme : SODIMM
    * Vitesse : 2667 MHz
    * Matériel reservé : 163 Mo

On peut aussi utiliser la commande PowerShell suivante :

```sh
(Get-CimInstance -ClassName Win32_PhysicalMemory | Measure-Object -Property Capacity -Sum).Sum / 1GB
16
```

Depuis Ubuntu :

```sh
$ free -h
              total        used        free      shared  buff/cache   available
Mem:          7.7Gi       574Mi       3.6Gi       2.0Mi       3.5Gi       6.8Gi
Swap:         2.0Gi          0B       2.0Gi
```

Hum, ce n'est pas assez précis..

```sh
$ free -m
              total        used        free      shared  buff/cache   available
Mem:           7873         574        3732           2        3566        7002
Swap:          2048           0        2048
```

On ne retrouve pas vraiment nos 16 GB!

```sh
$ cat /proc/meminfo
MemTotal:        8062644 kB
MemFree:         3822152 kB
MemAvailable:    7170564 kB
Buffers:          131012 kB
Cached:          3315812 kB
SwapCached:            0 kB
Active:           817908 kB
Inactive:        2940468 kB
Active(anon):        316 kB
Inactive(anon):   313724 kB
Active(file):     817592 kB
Inactive(file):  2626744 kB
...
```

8062644 kB = 7,7 GB

On ne retrouve donc pas côté Ubuntu l'intégralité de la mémoire matérielle, mais un petit peu moins que la moitié.

Cela semble indiquer que Windows n'alloue à son sous-système WSL qu'une partie de la mémoire disponible pour assurer ses propres besoins de fonctionnement.

# Exploitation des GPU ?

❓ Le rapport lscpu est infiniment plus détaillé que ce que l'on obtient côté Windows.

Question : j'ai deux processeurs graphiques, et j'ai entendu dire qu'ils pouvaient être exploités notamment pour faire de la data.

* GPU 0 - Intel(R) UHD Graphics 630
    * Mémoire partagée : 7,9 Go
* GPU 1 - NVIDIA GeForce RTX 2060
    * Mémoire dédiée : 6 Go
    * Mémoire partagée : 7,9 Go

Mais les rapports ne détaillent pas plus leurs capacités.

Comment en pratique les mobiliser par exemple dans le cadre d'un cluster Spark ?

📌 Pour utiliser les GPU dans le cadre d'un cluster Spark, nous pouvons utiliser la bibliothèque `Spark-GPU`, qui permet d'exécuter des calculs sur les GPU avec Spark.

C'est une approche à envisager si les tâches à exécuter peuvent tirer parti des performances des GPU. A réserver aux tâches Spark qui implémentent des opérations qui peuvent être accélérées par les GPU, comme des calculs matriciels, des opérations de filtrage ou de transformation d'images, etc.

Configuration du cluster Spark pour qu'il utilise ces GPU : il faut installer les pilotes GPU appropriés sur tous les nœuds du cluster, ainsi que les bibliothèques `CUDA` et `cuDNN`. Il faut également configurer la variable d'environnement `PYSPARK_SUBMIT_ARGS` pour inclure les options nécessaires pour activer la prise en charge GPU dans Spark.

Soumission des tâches Spark : il faut spécifier le nombre de GPU à utiliser pour chaque tâche à l'aide de la méthode `SparkContext.addPyFile` pour ajouter les fichiers Python contenant les fonctions GPU à utiliser.

### 📌 Complément de configuration client / serveur

Dans notre cas de figure où le client PySpark est sur l'OS Windows et le serveur (cluster) Spark sur l'OS Ubuntu/WSL, il convient de paramétrer les deux parties pour qu'elles puissent interagir. Voici les étapes à suivre pour modifier les paramètres de configuration par défaut :
1. Ouvrir le fichier de configuration `spark-defaults.conf` dans le dossier `conf` de l'installation Spark sur le nœud maître Ubuntu.
2. Ajoutez les lignes suivantes en modifiant les valeurs pour correspondre à la configuration :
```sh
spark.master spark://<adresse_ip_du_noeud_master_ubuntu>:7077
spark.driver.host <adresse_ip_du_noeud_windows>
```

master : 172.28.176.216
client : 172.28.176.1

3. Sur la machine Windows, ouvrir un terminal et configurer la variable d'environnement `PYSPARK_SUBMIT_ARGS` avec les options de configuration Spark précédentes :

```sh
set PYSPARK_SUBMIT_ARGS=--master spark://<adresse_ip_du_noeud_master_ubuntu>:7077 --conf spark.driver.host=<adresse_ip_du_noeud_windows>
```

Test en session, mais en l'occurrence sans intérêt 
```sh
$env:PYSPARK_SUBMIT_ARGS="--master spark://172.28.176.216:7077 --conf spark.driver.host=172.28.176.1"
```

Variable d'environnement système :
```sh
[Environment]::SetEnvironmentVariable("PYSPARK_SUBMIT_ARGS", "--master spark://172.28.176.216:7077 --conf spark.driver.host=172.28.176.1", "Machine")
```

4. Lancer PySpark en tapant `pyspark` dans le terminal. PySpark devrait maintenant communiquer avec le cluster Spark sur le nœud maître Ubuntu.

📌 Attention, il faudra peut-être aussi préciser le paramètre `spark.driver.bindAddress` qui spécifie l'adresse IP à laquelle le driver Spark doit être lié lorsqu'il s'exécute sur une machine à plusieurs interfaces réseau.

Par défaut, Spark essaie de deviner l'adresse IP en interrogeant la machine pour toutes les adresses IP disponibles et en sélectionnant la première adresse IP non locale qu'il trouve. Le paramètre `spark.driver.bindAddress` permet de désigner l'une de ces adresse en particulier.

Si Spark est exécuté en mode cluster, le paramètre `spark.driver.bindAddress` doit être défini sur l'adresse IP du nœud maître du cluster.